# OSNAP line Lagrangian particle tracking investigation of the cold/fresh blob


### The aim here is an investigation of the source of the cold/fresh blob which appeared in the near-surface Iceland Basin from about 2014. There are a couple of published explanations for the feature: 

- reduced heat/salt transport across RAPID array and northwards to subpolar N Atlantic over several years (Bryden et al. 2020?) or 
- increased contribution from Labrador Sea water flowing into subpolar North Atlantic rather than southwards (Holliday et al. 2020?). Driven I think by specific,  more intermittent, weather near Flemish Cap (though I need to read this again...)

The technique is to track particles back from the OSNAP line for 4 years and identify those which came from the Gulf Stream and those which came through the Labrador Sea. And see how that changes between from 2006 to 2017. (Elsewhere I have done backwards particle tracking for all years between 2006 and 2017. This supports the results shown here and adds some detail, but 2 years tracking isn't really long enough to identify pathways). I am also trying to quantify the volume/heat/salt transports along each route (not here yet though).

## Technical preamble

In [ ]:
# import matplotlib.colors as colors
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime, timedelta
import seaborn as sns
# from matplotlib.colors import ListedColormap
import cmocean as co
import pandas as pd
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import seawater as sw

from matplotlib import colors as c
from matplotlib import ticker
from xhistogram.xarray import histogram

In [ ]:
sns.set(style="darkgrid")

xr.set_options(keep_attrs=True)
np.warnings.filterwarnings('ignore')

sns.set_palette("colorblind")
xr.set_options(keep_attrs=True);

plt.rc('font', size=20) #controls default text size
plt.rc('axes', titlesize=20) #fontsize of the title
plt.rc('axes', labelsize=20) #fontsize of the x and y labels
plt.rc('xtick', labelsize=20) #fontsize of the x tick labels
plt.rc('ytick', labelsize=20) #fontsize of the y tick labels
plt.rc('legend', fontsize=20) #fontsize of the legend
plt.rc('savefig', dpi=300)       # higher res outputs

## Set up paths and read in trajectory data

In [ ]:
## Parameters
# Project path
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

# output figures path
figures_path = Path("reports/figures/ColdBlob/")

# Parcels track data file
path_data_tracks = Path('data/processed/tracks/osnap/') 
filename_tracks = [
                   'tracks_osnap_backward_200607_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200707_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200807_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200907_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201007_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201107_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201207_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201307_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201407_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201507_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201607_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201707_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200601_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200701_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200801_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200901_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201001_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201101_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201201_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201301_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201401_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201501_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201601_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201701_N2931_D1460_Rnd123.nc'
]

# model mask file
data_path = Path("data/external/iAtlantic/")
experiment_name = "VIKING20X.L46-KKG36107B"
mesh_mask_file = project_path / data_path / "mask" / experiment_name / "1_mesh_mask.nc"

#section lonlat file
sectionPath = Path('data/external/sections/')
sectionFilename = 'osnap_pos_wp.txt'
sectionname = 'osnap'
gsrsectionFilename = 'gsr_pos_wp.txt'

degree2km = 1.852*60.0


# some transport values specific to osnap runs
# randomly seeded 39995 particles, 19886 were in ocean points (the rest were land)

osnap_section_length = 3594572.87839    # m
osnap_subsection_length = 1121724.76238   # m
osnap_section_depth = 4000 # m over which particles launched
osnap_subsection_depth = 500 # m over which particles launched

osnap_subsection_ocean_area = osnap_subsection_length * osnap_subsection_depth * 2931.0 / 3119.0

particle_section_area = osnap_subsection_length * osnap_subsection_depth / 3119.0


## Load data

### mesh and masks

In [ ]:
mesh_mask = xr.open_dataset(mesh_mask_file)
mesh_mask = mesh_mask.squeeze()
mesh_mask = mesh_mask.set_coords(["nav_lon", "nav_lat", "nav_lev"])

bathy = mesh_mask.mbathy.rename("number of water filled points")

depth = (mesh_mask.e3t_0 * mesh_mask.tmask).sum("z")
# display(mesh_mask)

### section position data

In [ ]:
lonlat = xr.Dataset(pd.read_csv(project_path / sectionPath / sectionFilename,delim_whitespace=True))
lonlat['lon'] *= -1.0

In [ ]:
lonlat.lon.attrs['long_name']='Longitude'
lonlat.lat.attrs['long_name']='Latitude'
lonlat.lon.attrs['standard_name']='longitude'
lonlat.lat.attrs['standard_name']='latitude'
lonlat.lon.attrs['units']='degrees_east'
lonlat.lat.attrs['units']='degrees_north'

lonlat2mean= lonlat.rolling({'dim_0':2}).mean()

lonlatdiff = (lonlat.diff('dim_0'))

lonlatdiff = lonlatdiff.assign({'y':lonlatdiff['lat']*degree2km})
lonlatdiff = lonlatdiff.assign({'x':lonlatdiff['lon']*degree2km*np.cos(np.radians(lonlat2mean.lat.data[1:]))})
lonlatdiff=lonlatdiff.assign({'length':np.sqrt(lonlatdiff['x']**2+lonlatdiff['y']**2)})
lonlatdiff=lonlatdiff.assign({'costheta':lonlatdiff['x']/lonlatdiff['length']})
lonlatdiff=lonlatdiff.assign({'sintheta':lonlatdiff['y']/lonlatdiff['length']})

total_length = lonlatdiff.length.sum().data 
total_osnap_length = lonlatdiff.length[0:12].sum().data;  # exclude section across UK - just there for testing north/south

length_west = xr.concat((xr.DataArray([0],dims=("dim_0"),coords={"dim_0": [0]}),lonlatdiff.length.cumsum()),dim='dim_0')

### tracks

In [ ]:
for i,filename in enumerate(filename_tracks):
    dst=xr.open_dataset(project_path / path_data_tracks / filename)
    if i == 0:
        ds=dst
    else:
        ds=xr.concat([ds,dst],dim='traj')


#### Subset tracks by OSNAP line cross longitude and depth range

In [ ]:
lonRange=[-30,-19]
depthRange=[0,500]

In [ ]:
ds = ds.where(ds.isel(obs=0).lon > lonRange[0]).where(ds.isel(obs=0).lon < lonRange[1])
ds = ds.where(ds.isel(obs=0).z > depthRange[0]).where(ds.isel(obs=0).z < depthRange[1])
ds = ds.dropna('traj', how='all')
    

#### Velocity conversions from degrees lat/lon per second to m/s

In [ ]:
ds=ds.assign({'uvel_ms':ds.uvel * degree2km * 1000.0 * np.cos(np.radians(ds.lat))})
ds=ds.assign({'vvel_ms':ds.vvel * degree2km * 1000.0})

#### Extract fields on osnap line (obs=0). Find 'initial' velocities normal to the section.

These are the positions we 'initialised' the particles with, but we ran particles backwards in time so these  represent the final positions, on the osnap line.

In [ ]:
ds_init = ds.isel(obs=0)

In [ ]:
ds_init = ds_init.assign({'section_index':xr.DataArray(np.searchsorted(lonlat.lon,ds_init.lon)-1,dims='traj')})
ds_init = ds_init.assign({'u_normal':ds_init.vvel_ms * lonlatdiff.costheta[ds_init.section_index].data - ds_init.uvel_ms * lonlatdiff.sintheta[ds_init.section_index].data})
ds_init = ds_init.assign({'u_along':ds_init.vvel_ms * lonlatdiff.sintheta[ds_init.section_index].data + ds_init.uvel_ms * lonlatdiff.costheta[ds_init.section_index].data})


#### Find along-section distances of initial points

In [ ]:
ds_init = ds_init.assign({'x':xr.DataArray(length_west[ds_init.section_index] + lonlatdiff.length[ds_init.section_index]*
                              (ds_init.lon - lonlat.lon[ds_init.section_index])/lonlatdiff.lon[ds_init.section_index],dims='traj')})

### volume, temperature and salt transports along track

In [ ]:
# at osnap line
ds_init = ds_init.assign({'particle_vol':(ds_init.salt/ds_init.salt) * particle_section_area/1.0e06})
ds_init = ds_init.assign({'vol_trans_normal':ds_init.u_normal * particle_section_area/1.0e06})
ds_init = ds_init.assign({'temp_transport':ds_init.temp * ds_init.vol_trans_normal})
ds_init = ds_init.assign({'salt_transport':ds_init.salt * ds_init.vol_trans_normal})
ds_init = ds_init.assign({'depth_transport':ds_init.z * ds_init.vol_trans_normal})
ds_init = ds_init.assign({'lon_transport':ds_init.lon * ds_init.vol_trans_normal})

# full track assuming alongtrack volume transport conserved following particle
ds = ds.assign({'temp_transport':ds.temp * ds_init.vol_trans_normal})
ds = ds.assign({'salt_transport':ds.salt * ds_init.vol_trans_normal})
ds = ds.assign({'vol_transport':ds_init.vol_trans_normal * ds.salt/ds.salt}) # trick to get full array dimensions

## Plot section

In [ ]:
sns.set(style="whitegrid")
central_lon, central_lat = -30, 55
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.Orthographic(central_lon, central_lat)})
extent = [-60, 0, 40, 70]
ax.set_extent(extent)
ax.gridlines()
ax.coastlines(resolution='50m')

lonlat.plot.scatter(ax=ax,transform=ccrs.PlateCarree(),x='lon',y='lat')
lonlat2mean.plot.scatter(ax=ax,transform=ccrs.PlateCarree(),x='lon',y='lat');

## plot tracks by source region

In [ ]:
# range of stations from west to east, stations 0-12. Python indexing.

def plotTracksCartopy(ds1,cmap=co.cm.tempo_r,title='',fig_path = Path.cwd(),fileroot='',date = ''):
    
    central_lon, central_lat = -45, 47.5
    sns.set(style="whitegrid")
    fig, ax = plt.subplots(figsize = (12,8),subplot_kw={'projection': ccrs.Orthographic(central_lon, central_lat)})
    extent = [-80, -10, 25, 70]
    ax.set_extent(extent)
    ax.gridlines()
    ax.coastlines(resolution='50m')
    
    dsmask = ds1

    pcm = ax.scatter(
        dsmask.lon.data.flatten(),
        dsmask.lat.data.flatten(),
        3,
#         (dsmask.time.data.flatten()-ds.time.isel(obs=0,traj=0).data).astype('timedelta64[D]')
        mdates.date2num(dsmask.time.data.flatten())
#         ,cmap= cmap
        ,zorder=2
        ,transform=ccrs.PlateCarree()
    #   ,alpha=0.3
    )
    cb = fig.colorbar(pcm,ax=ax,shrink=0.8,label = "date")
    loc = mdates.MonthLocator(bymonth=range(1,13,3))
    cb.ax.yaxis.set_major_locator(loc)
    cb.ax.yaxis.set_major_formatter(mdates.ConciseDateFormatter(loc))
#     cb.ax.tick_params(labelsize=20)


    depth.isel(y=slice(1000, 2499), x=slice(0, 2404)).plot.contour(ax=ax,transform=ccrs.PlateCarree(),
        x="nav_lon", y="nav_lat", colors = 'grey', levels = [200,800,1500,2000,2500,3500],zorder=1
    );
    ax.scatter(
        dsmask.lon.isel(obs=0).data.flatten(),
        dsmask.lat.isel(obs=0).data.flatten(),2,zorder=5,
        transform=ccrs.PlateCarree()
    )
    ax.plot(np.linspace(-58.2,-40,num=20),np.linspace(52,65,num=20),
        zorder=5,color='C2',
        transform=ccrs.PlateCarree()
    )
    ax.plot(np.linspace(-80,-58.2,num=20),np.linspace(52,52,num=20),
        zorder=5,color='C2',
        transform=ccrs.PlateCarree()
    )
    ax.plot(np.linspace(-60,-100,num=20),np.linspace(33,33,num=20),
        zorder=5,color='C2',
        transform=ccrs.PlateCarree()
    )
    
    title=ax.set_title(title)
    
    filenameroot = fig_path / str('plotTracks' + fileroot + date)

#     plt.savefig(filenameroot.with_suffix('.eps'), bbox_extra_artists=(title,), bbox_inches='tight')
    plt.savefig(filenameroot.with_suffix('.png'), bbox_extra_artists=(title,), bbox_inches='tight')
#     plt.savefig(filenameroot.with_suffix('.pdf'), bbox_extra_artists=(title,), bbox_inches='tight')


    return


In [ ]:
def apply_left_of_line(ds, lon_1, lon_2, lat_1, lat_2):
    '''Apply an area crossing criterion.
    
    Larvae in ds selected while they are in a selected area.
    '''
    # particles are selected if they pass through given area.
    position =  ((lon_2 -lon_1) * (ds.lat - lat_1) - 
                     (ds.lon - lon_1) * (lat_2 - lat_1))
                        
    return position > 0.0, position < 0

#### from Labrador sea or from Gulf Stream

In [ ]:
# labrador sea
ds_in1, ds_notin1 = apply_left_of_line(ds,-75,-40,40,65)
ds_in2, ds_notin2 = apply_left_of_line(ds,-95,-60,52,52)
ds_lab_sea_in = ds_in1*ds_in2

# gulf stream
ds_in, ds_notin = apply_left_of_line(ds,-60,-100,33,33)
ds_60w_in = ds_in


In [ ]:
# particles on osnap line
ds_init_labsea = ds_init.where(ds_lab_sea_in.max("obs"))
ds_init_60w = ds_init.where(ds_60w_in.max("obs"))
ds_init_other = ds_init.where(ds_60w_in.max("obs")==False).where(ds_lab_sea_in.max("obs")==False)

# particles throughout tracking timespan
ds_lab_sea = ds.where(ds_lab_sea_in.max("obs"))
ds_60w = ds.where(ds_60w_in.max("obs"))
ds_other = ds.where(ds_lab_sea_in.max("obs")==False).where(ds_60w_in.max("obs")==False) 

In [ ]:
t=[]

for name,group in ds_init.groupby("time"):
    t.append(name)  
    
display(t)


### Particles reaching OSNAP section, 20-30W 0-500 m depth, from Labrador Sea.

It is difficult to see any change in the volume transort of water on this route from these plots, but there is a change in the pattern, with many fewer of the 2017 particles taking the southern loop down the east coast of Canada/USA than the 2006 particles.

This supports the Holliday et al idea of different routes out of the Labrador Sea influencing temperature/salinity in near-surface Iceland Basin. It also suggests a possible negative link between near-surface temperature in Iceland Basin and Atlantic coast of NE USA/Canada.

In [ ]:
date = str(t[4].astype('datetime64[D]'))
display(date)

In [ ]:
test = ds_lab_sea.where(ds_lab_sea_in.cumsum(dim='obs') < 1)
ds_select = test.where(test.isel(obs=0).time == t[4]).dropna('traj', how='all')

In [ ]:
plotTracksCartopy(ds_select,cmap=co.cm.tempo_r,title='From Labrador Sea '+date,fig_path = project_path / figures_path, fileroot='Lab',date=date)

In [ ]:
date = str(t[16].astype('datetime64[D]'))
display(date)

In [ ]:
test = ds_lab_sea.where(ds_lab_sea_in.cumsum(dim='obs') < 1)
ds_select = test.where(test.isel(obs=0).time == t[16]).dropna('traj', how='all')

In [ ]:
plotTracksCartopy(ds_select,cmap=co.cm.tempo_r,title='From Labrador Sea '+date,fig_path = project_path / figures_path, fileroot='Lab',date=date)

### Particles reaching OSNAP section, 20-30W 0-500 m depth, from Gulf Stream.

This shows with fewer of the 2017 particles reaching the OSNAP line from the Gulf Stream (tested crossing 33N) than 2006 particles.

I think this supports the ideas of Bryden et al (2020?), with cooling/freshening in eastern subpolar gyre following reduced surface heat/salt transport northwards from Gulf Stream to subpolar N Atlantic.

In [ ]:
date = str(t[4].astype('datetime64[D]'))
display(date)

In [ ]:
test = ds_60w.where(ds_60w_in.cumsum(dim='obs') < 1)
ds_select = test.where(test.isel(obs=0).time == t[4]).dropna('traj', how='all')

In [ ]:
plotTracksCartopy(ds_select,cmap=co.cm.tempo_r,title='From Gulf Stream '+date,fig_path = project_path / figures_path, fileroot='GS',date=date)

In [ ]:
date = str(t[16].astype('datetime64[D]'))
display(date)

In [ ]:
test = ds_60w.where(ds_60w_in.cumsum(dim='obs') < 1)
ds_select = test.where(test.isel(obs=0).time == t[16]).dropna('traj', how='all')

In [ ]:
plotTracksCartopy(ds_select,cmap=co.cm.tempo_r,title='From Gulf Stream '+date,fig_path = project_path / figures_path, fileroot='GS',date=date)

### Other particles reaching OSNAP section, 20-30W 0-500 m depth.

However, there are a lot of particles for which running backwards for 4 years isn't long enough to determine whether they came from the Labrador Sea or the Gulf Stream, as they spent the whole of that time in the N Atlantic (N of 33N, not in Labrador Sea).

In [ ]:
date = str(t[4].astype('datetime64[D]'))
display(date)

In [ ]:
ds_select = ds_other.where(ds_other.isel(obs=0).time == t[4]).dropna('traj', how='all')

In [ ]:
plotTracksCartopy(ds_select,cmap=co.cm.tempo_r,title='From other '+date,fig_path = project_path / figures_path, fileroot='Other',date=date)

In [ ]:
date = str(t[16].astype('datetime64[D]'))
display(date)

In [ ]:
ds_select = ds_other.where(ds_other.isel(obs=0).time == t[16]).dropna('traj', how='all')

In [ ]:
plotTracksCartopy(ds_select,cmap=co.cm.tempo_r,title='From other '+date,fig_path = project_path / figures_path, fileroot='Other',date=date)

In [ ]:
# conda list